# Series Expansions: Fourier

Series expansions are used in a variety of circumstances:
- When we need a tractable approximation to some ugly equation
- To transform between equivalent ways of looking at a problem (e.g. time domain vs frequency domain)
- When they are (part of) a solution to a particular class of differential equation

For approximations, there is an important divide between getting the best fit *near a point* (e.g. Taylor series) and getting the best fit *over an interval* (e.g. Fourier series). This notebook deals with the latter; there is a separate notebook for Taylor expansions, and others for Bessel, Legendre, etc.

## Fitting over an interval

What is the best (tractable) series approximating my function across some range of values? What matters is an overall best fit (e.g. least-squares deviation) across the range, and we can't tolerate wild divergences as with the Taylor series.

There are various series which are useful in different contexts, but a common property is that the terms are *orthogonal* over some interval $[-L,L]$. If $f(t)$ is a real-valued function their *inner product* is defined as

$$ \langle f(m t),f(n t) \rangle   \colon =\int _{-L}^L f(m t) f(n t) \,  dt $$

For orthogonal functions, this is non-zero if $m=n$ and zero if $m \ne n$. If the inner product is $\delta_{mn}$ (the Kronecker delta), the fuctions are said to be orthonormal.

# Differential Equations

The ODE $y'' + n^2 y = 0$ is solved by all the period functions $\sin(n x)$, $\cos(n x)$ and $e^{\pm i n x}$. There is thus a close analogy to functions such as Bessel and Legendre, though sine and cosine have become much more familar to most of us for other (geometric) reasons. They have the nice property of evenly-spaced zeros, unlike Bessel functions. For example, $\sin(x)=0$ for $x = n \pi$ where n is any integer.

The use of sin/cos or complex exponentials is also exceptionally familiar in series expansions, mainly because they are so useful in engineering and communications.

## Fourier Series and Fourier Analysis

A periodic function $f$ of period $2L$ can be approximated by a Fourier Series of sines and cosines:

$$ f(t) = \frac{a_0}{2} + \sum _{n \ge 1} a_ n \cos \frac{n \pi t}{L} + \sum _{n \ge 1} b_ n \sin \frac{n \pi t}{L}  $$

To find the coefficients:
$$
\begin{align*}
	\frac{a_0}{2} &= \displaystyle \frac{1}{2L} \int _{-L}^{L} f(t) \,  dt = \frac{\langle f(t), 1 \rangle }{\langle 1, 1\rangle }\\[6pt]
	a_ n& = \frac{1}{L} \int _{-L}^{L} f(t) \cos \frac{n \pi t}{L} \,  dt = \frac{\langle f(t),\cos \left(\frac{n \pi }{L} t\right)\rangle }{\langle \cos \left(\frac{n \pi }{L} t\right), \cos \left(\frac{n \pi }{L} t\right)\rangle } \\[10pt]
	b_ n &= \displaystyle \frac{1}{L} \int _{-L}^{L} f(t) \sin \frac{n \pi t}{L} \,  dt = \frac{\langle f(t),\sin \left(\frac{n \pi }{L} t\right)\rangle }{\langle \sin \left(\frac{n \pi }{L} t\right), \sin \left(\frac{n \pi }{L} t\right)\rangle }
\end{align*}
$$

Equivalently, we can express the Fourier Series as complex exponentials:

$$ f\left(t\right) = \sum _{n = -\infty }^{\infty } c_{n} e^{i n t}, \qquad c_{n} \colon =\frac{a_{n} - i b_{n}}{2} \quad \text{ and } \quad c_{-n} \colon =\bar{c}_{n} = \frac{a_{n} + i b_{n}}{2} $$

Real-world situations tend not to give infinitely periodic functions, so Fourier Analysis can be thought of as the limit as $L$ goes to infinity of a periodic signal of period $2L$. As $L$ increases, the spacing between the frequencies in our sum are approaching zero. This turns the sum into an integral in the limit, and we have the equations:
 
$$ f(t)  = \int _{-\infty }^{\infty } \widehat{f}\left(k\right)e^{ i k t} \,  dk  \quad \text{where} \quad \widehat{f} = \frac{1}{2\pi }\int _{-\infty }^{\infty } f\left(t\right)e^{- i k t} \,  dt $$

We call $\widehat{f}$ the **Fourier transform** of $f(t)$.

In [1]:
# TODO - add graphical example

### Discrete Fourier transforms

The mathematics of Fourier analysis goes back to the early 19th century, but its use has exploded in the last few decades. A couple of factors collided to drive this:

- An efficient Fast Fourier Transform (FFT) algorithm, developed in the 1960s and implemented in both software and specialist hardware
- The spread of digital technology, for audio, video and many other sorts of discretized signals. These are all perfect inputs for FFT.

FFT gets away from complicated integrals and replaces them with a series of simple multiplications and additions. This gives a computation time of $\mathcal{O}(N \log N)$ for a signal with $N$ data points. Fast, as the name suggests! And your cellphone is doing millions of these calculations whenever you use if (for anything at all).

**TODO** - add graphical example

<a id='bessel'></a>

<a id='refs'></a>

## References

Boas, "Mathematical methods in the physical sciences"